##                                                   BigMart Sales Prediction

In [1]:
##########################
# IMPORTING LIBRARIES
##########################

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error

from sklearn.preprocessing import LabelEncoder

In [2]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
df= pd.read_csv("Files/model_dataset.csv")

In [4]:
df.head()

Outlet_Identifier  Outlet_Age  Item_Weight  Item_Visibility  Item_MRP  \
0                  9          14         9.30         0.016047  249.8092   
1                  3           4         5.92         0.019278   48.2692   
2                  9          14        17.50         0.016760  141.6180   
3                  0          15        19.20         0.000000  182.0950   
4                  1          26         8.93         0.000000   53.8614   

   Item_Visibility_Log  Item_Outlet_Sales  Outlet_Size_High  \
0            -3.612308          3735.1380                 0   
1            -3.612308           443.4228                 0   
2            -3.612308          2097.2700                 0   
3            -3.612308           732.3800                 0   
4            -3.612308           994.7052                 1   

   Outlet_Size_Medium  Outlet_Size_Small  ...  Outlet_Location_Type_Tier 2  \
0                   1                  0  ...                            0   
1                   1                  0  ...                            0   
2                   1                  0  ...                            0   
3                   0                  1  ...                            0   
4                   0                  0  ...                            0   

   Outlet_Location_Type_Tier 3  Item_Fat_Content_Low Fat  \
0                            0                         1   
1                            1                         0   
2                            0                         1   
3                            1                         0   
4                            1                         1   

   Item_Fat_Content_Regular  Visibility_Bucket_Low  Visibility_Bucket_Medium  \
0                         0                      1                         0   
1                         1                      1                         0   
2                         0                      1                         0   
3                         1                      0                         0   
4                         0                      0                         0   

   Visibility_Bucket_High  Item_Category_Drinks  Item_Category_Food  \
0                       0                     0                   1   
1                       0                     1                   0   
2                       0                     0                   1   
3                       0                     0                   1   
4                       0                     0                   0   

   Item_Category_Non-Consumables  
0                              0  
1                              0  
2                              0  
3                              0  
4                              1  

[5 rows x 32 columns]

In [5]:
df.shape

(8523, 32)

In [86]:
final_cat_features= ['Item_Fat_Content', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type',
                     'Outlet_Type', 'Item_Category', 'Outlet_Age', 'MRP_Bucket', 'Weight_Bucket', 'Visibility_Bucket'
                    ]
final_num_features= ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Visibility_Log']

### Model Experimentations

In [12]:
X= df.drop(columns= 'Item_Outlet_Sales')
y= df['Item_Outlet_Sales']

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size= 0.3, random_state= 42)
X_train.shape, y_train.shape

((5966, 31), (5966,))

In [22]:
def ModelResults(model, params= {}):
    m= model(**params)
    m.fit(X_train, y_train)
    pred_train= m.predict(X_train)
    pred_test= m.predict(X_test)
    
    r2_train= r2_score(y_train, pred_train)
    rmse_train= root_mean_squared_error(y_train, pred_train)
    
    r2_test= r2_score(y_test, pred_test)
    rmse_test= root_mean_squared_error(y_test, pred_test)
    
    df= pd.DataFrame([{
        'model': model.__name__, 
        'R2 Train': r2_train,
        'RMSE Train': rmse_train,
        'R2 Test': r2_test,
        'RMSE Test': rmse_test 
    }])
    return df

In [24]:
ModelResults(XGBRegressor)

model  R2 Train  RMSE Train   R2 Test   RMSE Test
0  XGBRegressor   0.87381  610.901093  0.521753  1157.39636

In [30]:
df_res= pd.DataFrame()
for model in [Lasso, RandomForestRegressor, XGBRegressor, LGBMRegressor, CatBoostRegressor]:
    df_model= ModelResults(model)
    df_res= pd.concat([df_res, df_model], axis= 0)

df_res

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000089 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1061
[LightGBM] [Info] Number of data points in the train set: 5966, number of used features: 31
[LightGBM] [Info] Start training from score 2204.882521
Learning rate set to 0.054292
0:	learn: 1666.3060931	total: 1.01ms	remaining: 1.01s
1:	learn: 1620.1407865	total: 2.08ms	remaining: 1.03s
2:	learn: 1575.2091923	total: 3.05ms	remaining: 1.01s
3:	learn: 1535.8954604	total: 3.98ms	remaining: 992ms
4:	learn: 1497.2796687	total: 4.88ms	remaining: 970ms
5:	learn: 1462.1420331	total: 5.76ms	remaining: 954ms
6:	learn: 1429.0952658	total: 6.63ms	remaining: 941ms
7:	learn: 1398.5414827	total: 7.5ms	remaining: 931ms
8:	learn: 1370.6570814	total: 8.4ms	remaining: 925m

model  R2 Train   RMSE Train   R2 Test    RMSE Test
0                  Lasso  0.560107  1140.593413  0.568279  1099.657903
0  RandomForestRegressor  0.936975   431.731195  0.554671  1116.853996
0           XGBRegressor  0.873810   610.901093  0.521753  1157.396360
0          LGBMRegressor  0.732057   890.182586  0.573950  1092.411074
0      CatBoostRegressor  0.767487   829.242046  0.582928  1080.840200

In [32]:
#df_imp= pd.DataFrame({'features': X.columns, 'importance': (xgb.feature_importances_)*100})
#df_imp.sort_values('importance', ascending= False)

#### Hyper Parameter Tuning for CatBoost Model

In [34]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

X_train_tune, X_val_tune, y_train_tune, y_val_tune=  train_test_split(X, y, test_size= 0.3, random_state= 32)

# Define the objective function
def objective(params):
    model = CatBoostRegressor(
        random_state = 42,
        #iterations= int(params['iterations']),
        iterations= 500,
        depth= int(params['depth']),
        border_count= int(params['border_count']),
    
    # Float parameters
    learning_rate= params['learning_rate'],
    l2_leaf_reg= params['l2_leaf_reg'],
    bagging_temperature= params['bagging_temperature'],
    random_strength= params['random_strength'],
    subsample= params['subsample']
    )
    
    model.fit(X_train_tune, y_train_tune)
    y_pred = model.predict(X_val_tune)
    mse = mean_squared_error(y_val_tune, y_pred)
    return {'loss': mse, 'status': STATUS_OK}

# Define the hyperparameter space
space = {
    # Integer parameters
    #'iterations': hp.quniform('iterations', 500, 2000, 50),
    'depth': hp.quniform('depth', 4, 10, 1),
    'border_count': hp.quniform('border_count', 32, 255, 1),
    
    # Float parameters
    'learning_rate': hp.loguniform('learning_rate', -5, 0),  # ~0.006 to 1
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg', -5, 2),
    'bagging_temperature': hp.uniform('bagging_temperature', 0, 1),
    'random_strength': hp.uniform('random_strength', 0, 1),
    'subsample': hp.uniform('subsample', 0.6, 1.0)
}

# Run the hyperparameter optimization
trials = Trials()
best = fmin(fn= objective, space= space, algo=tpe.suggest, max_evals= 100, trials= trials, verbose= False)


0:	learn: 1621.9166510	total: 954us	remaining: 476ms
1:	learn: 1550.3360712	total: 3.18ms	remaining: 793ms
2:	learn: 1485.8408738	total: 4.6ms	remaining: 761ms
3:	learn: 1431.5393237	total: 5.51ms	remaining: 683ms
4:	learn: 1383.3749678	total: 6.43ms	remaining: 637ms
5:	learn: 1340.4994618	total: 7.87ms	remaining: 648ms
6:	learn: 1301.8558784	total: 9.31ms	remaining: 656ms
7:	learn: 1268.7079737	total: 10.7ms	remaining: 659ms
8:	learn: 1241.7161564	total: 12.1ms	remaining: 661ms
9:	learn: 1217.6382107	total: 13.6ms	remaining: 666ms
10:	learn: 1196.8246919	total: 15ms	remaining: 668ms
11:	learn: 1178.6808406	total: 16.4ms	remaining: 667ms
12:	learn: 1162.5891335	total: 18.3ms	remaining: 687ms
13:	learn: 1148.7316230	total: 19.8ms	remaining: 686ms
14:	learn: 1135.9079936	total: 21.2ms	remaining: 684ms
15:	learn: 1125.3849935	total: 22.6ms	remaining: 683ms
16:	learn: 1116.2885674	total: 24ms	remaining: 683ms
17:	learn: 1108.7568216	total: 25.4ms	remaining: 681ms
18:	learn: 1102.2936550	to

In [35]:
print(best)

{'bagging_temperature': 0.28917112832006814, 'border_count': 49.0, 'depth': 4.0, 'l2_leaf_reg': 4.3815888753770675, 'learning_rate': 0.011399763616104164, 'random_strength': 0.9885978286212234, 'subsample': 0.9263982097755638}


In [84]:
X_train

Outlet_Identifier  Outlet_Age  Item_Weight  Item_Visibility  Item_MRP  \
1921                  0          15     7.720000         0.147904  116.6466   
655                   8          16    19.750000         0.014301  103.0332   
2229                  5          28    12.857645         0.054221  129.1310   
2537                  7          11    19.350000         0.118342  222.6088   
3528                  1          26    18.350000         0.092150  184.8266   
...                 ...         ...          ...              ...       ...   
5734                  0          15     9.395000         0.286345  139.1838   
5191                  2           6    15.600000         0.117575   75.6670   
5390                  7          11    17.600000         0.018944  237.3590   
860                   2           6    20.350000         0.054363  117.9466   
7270                  8          16    16.350000         0.016993   95.7410   

      Item_Visibility_Log  Outlet_Size_High  Outlet_Size_Medium  \
1921            -1.911190                 0                   0   
655             -3.612308                 0                   0   
2229            -2.914694                 0                   1   
2537            -2.134174                 0                   0   
3528            -2.384338                 1                   0   
...                   ...               ...                 ...   
5734            -1.250558                 0                   0   
5191            -2.140683                 0                   0   
5390            -3.612308                 0                   0   
860             -2.912077                 0                   0   
7270            -3.612308                 0                   0   

      Outlet_Size_Small  MRP_Bucket_Low  ...  Outlet_Location_Type_Tier 2  \
1921                  1               0  ...                            0   
655                   1               0  ...                            0   
2229                  0               0  ...                            0   
2537                  1               0  ...                            1   
3528                  0               0  ...                            0   
...                 ...             ...  ...                          ...   
5734                  1               0  ...                            0   
5191                  1               0  ...                            1   
5390                  1               0  ...                            1   
860                   1               0  ...                            1   
7270                  1               0  ...                            0   

      Outlet_Location_Type_Tier 3  Item_Fat_Content_Low Fat  \
1921                            1                         0   
655                             0                         1   
2229                            1                         0   
2537                            0                         1   
3528                            1                         0   
...                           ...                       ...   
5734                            1                         0   
5191                            0                         1   
5390                            0                         1   
860                             0                         1   
7270                            0                         1   

      Item_Fat_Content_Regular  Visibility_Bucket_Low  \
1921                         1                      0   
655                          0                      1   
2229                         1                      0   
2537                         0                      0   
3528                         1                      0   
...                        ...                    ...   
5734                         1                      0   
5191                         0                      0   
5390                         0                      1   


In [90]:
#best['iterations'] = int(best['iterations'])
best['depth'] = int(best['depth'])
best['border_count'] = int(best['border_count'])

# ---------------------------
# 5. Train Final Model with Best Params
# ---------------------------
best_model = CatBoostRegressor(**best, random_state=42, iterations= 500)
best_model.fit(X_train, y_train)

# Final Evaluation
y_pred = best_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)
print(f"Final RMSE after Optuna tuning: {rmse:.2f}")

print("R2 Score Train: ", r2_score(y_train, best_model.predict(X_train)))
print("R2 Score: ", r2_score(y_test, y_pred))

0:	learn: 1709.3278047	total: 2.69ms	remaining: 1.34s
1:	learn: 1699.4811414	total: 4.08ms	remaining: 1.01s
2:	learn: 1689.2313288	total: 4.86ms	remaining: 806ms
3:	learn: 1679.1498189	total: 6.13ms	remaining: 760ms
4:	learn: 1669.2082408	total: 6.85ms	remaining: 678ms
5:	learn: 1659.4302171	total: 7.56ms	remaining: 622ms
6:	learn: 1650.2526182	total: 8.25ms	remaining: 581ms
7:	learn: 1640.7647679	total: 8.9ms	remaining: 547ms
8:	learn: 1631.5011689	total: 9.61ms	remaining: 524ms
9:	learn: 1622.3350775	total: 10.3ms	remaining: 504ms
10:	learn: 1613.3983200	total: 10.9ms	remaining: 484ms
11:	learn: 1604.5064018	total: 11.7ms	remaining: 476ms
12:	learn: 1595.7634823	total: 12.5ms	remaining: 469ms
13:	learn: 1587.1150820	total: 13.6ms	remaining: 472ms
14:	learn: 1578.7823413	total: 14.4ms	remaining: 465ms
15:	learn: 1571.2070950	total: 15.2ms	remaining: 459ms
16:	learn: 1563.1060156	total: 16.2ms	remaining: 460ms
17:	learn: 1555.0125393	total: 17.2ms	remaining: 460ms
18:	learn: 1547.12049

### Test Predictions

In [50]:
df_test= pd.read_csv("Files/test_AbJTz2l.csv")

In [52]:
df_test.head()

Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility    Item_Type  \
0           FDW58       20.750          Low Fat         0.007565  Snack Foods   
1           FDW14        8.300              reg         0.038428        Dairy   
2           NCN55       14.600          Low Fat         0.099575       Others   
3           FDQ58        7.315          Low Fat         0.015388  Snack Foods   
4           FDY38          NaN          Regular         0.118599        Dairy   

   Item_MRP Outlet_Identifier  Outlet_Establishment_Year Outlet_Size  \
0  107.8622            OUT049                       1999      Medium   
1   87.3198            OUT017                       2007         NaN   
2  241.7538            OUT010                       1998         NaN   
3  155.0340            OUT017                       2007         NaN   
4  234.2300            OUT027                       1985      Medium   

  Outlet_Location_Type        Outlet_Type  
0               Tier 1  Supermarket Type1  
1               Tier 2  Supermarket Type1  
2               Tier 3      Grocery Store  
3               Tier 2  Supermarket Type1  
4               Tier 3  Supermarket Type3

In [54]:
df_test.shape

(5681, 11)

In [56]:
df_test.isna().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

#### Test data preprocessing and Feature Engg.

In [59]:
def Preprocessing(dataset):
    dataset['Item_Weight'].fillna(dataset['Item_Weight'].mean(), inplace= True)
    dataset['Outlet_Size'] = dataset.groupby('Outlet_Type')['Outlet_Size'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'}, inplace= True)
    return dataset

# Define mapping
food_items = ['Dairy', 'Meat', 'Fruits and Vegetables', 'Snack Foods',
              'Breads', 'Breakfast', 'Frozen Foods', 'Canned', 'Starchy Foods']
drink_items = ['Soft Drinks', 'Hard Drinks']
non_consumables = ['Household', 'Health and Hygiene', 'Others']

def categorize_item(item):
    if item in food_items:
        return 'Food'
    elif item in drink_items:
        return 'Drinks'
    else:
        return 'Non-Consumables'

def FeatureEng(dataset):
    dataset['Item_Category'] = dataset['Item_Type'].apply(categorize_item)
    dataset['Outlet_Age']= 2013- dataset['Outlet_Establishment_Year']
    dataset['MRP_Bucket'] = pd.cut(dataset['Item_MRP'], 
                          bins=[0, 65, 135, 200, 270], 
                          labels=['Low', 'Medium', 'High', 'Very_High'])

    dataset['Weight_Bucket']= pd.cut(dataset['Item_Weight'], 
                          bins=[0, 12, 13, 22], 
                          labels=['Low', 'Mode', 'High'])

    dataset['Visibility_Bucket']= pd.cut(dataset['Item_Visibility'], 
                          bins=[0, 0.03, 0.08, 0.4], 
                          labels=['Low', 'Medium', 'High'])

    dataset['Item_Visibility_Log']= np.log(dataset['Item_Visibility'])
    dataset['Item_Visibility_Log']= np.where(dataset['Item_Visibility_Log']< np.quantile(dataset['Item_Visibility_Log'], 0.25), 
                                    np.quantile(dataset['Item_Visibility_Log'], 0.25), dataset['Item_Visibility_Log'])

    return dataset
    

df_test= Preprocessing(df_test)
df_test= FeatureEng(df_test)

In [61]:
df_test_final= df_test[final_cat_features+ final_num_features]

In [63]:
df_test_final.columns

Index(['Item_Fat_Content', 'Outlet_Identifier', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type', 'Item_Category', 'Outlet_Age',
       'MRP_Bucket', 'Weight_Bucket', 'Visibility_Bucket', 'Item_Weight',
       'Item_Visibility', 'Item_MRP', 'Item_Visibility_Log'],
      dtype='object')

In [65]:
label= LabelEncoder()
df_test_final['Outlet_Identifier']= label.fit_transform(df_test_final['Outlet_Identifier'])

df_test_final= pd.get_dummies(df_test_final, columns= list(set(final_cat_features)- set(['Outlet_Identifier', 'Outlet_Age'])), dtype= int)

In [67]:
print(best)

{'bagging_temperature': 0.28917112832006814, 'border_count': 49, 'depth': 4, 'l2_leaf_reg': 4.3815888753770675, 'learning_rate': 0.011399763616104164, 'random_strength': 0.9885978286212234, 'subsample': 0.9263982097755638}


#### Final Modelling and Submission File

In [70]:
final_model = CatBoostRegressor(**best, random_state= 42, iterations= 100)
final_model.fit(X, y)

0:	learn: 1695.7304009	total: 1.57ms	remaining: 156ms
1:	learn: 1685.7831998	total: 3.12ms	remaining: 153ms
2:	learn: 1675.5569699	total: 4.1ms	remaining: 133ms
3:	learn: 1665.4424398	total: 5.33ms	remaining: 128ms
4:	learn: 1655.4489157	total: 6.36ms	remaining: 121ms
5:	learn: 1645.7274104	total: 7.35ms	remaining: 115ms
6:	learn: 1636.5651889	total: 8.53ms	remaining: 113ms
7:	learn: 1627.1862309	total: 9.58ms	remaining: 110ms
8:	learn: 1617.9614979	total: 10.7ms	remaining: 108ms
9:	learn: 1608.7345800	total: 11.8ms	remaining: 106ms
10:	learn: 1599.7908174	total: 12.7ms	remaining: 103ms
11:	learn: 1590.9984730	total: 13.6ms	remaining: 100ms
12:	learn: 1582.2778931	total: 14.5ms	remaining: 97ms
13:	learn: 1573.6170997	total: 15.4ms	remaining: 94.3ms
14:	learn: 1565.2666743	total: 16.3ms	remaining: 92.2ms
15:	learn: 1556.9995060	total: 17.2ms	remaining: 90.3ms
16:	learn: 1549.0817159	total: 18.1ms	remaining: 88.3ms
17:	learn: 1541.0434964	total: 18.9ms	remaining: 86ms
18:	learn: 1533.165

In [72]:
final_pred= final_model.predict(df_test_final[X.columns])

In [74]:
df_sub= pd.read_csv("Files/sample_submission_8RXa3c6.csv")

In [76]:
df_sub

Item_Identifier Outlet_Identifier  Item_Outlet_Sales
0              FDW58            OUT049               1000
1              FDW14            OUT017               1000
2              NCN55            OUT010               1000
3              FDQ58            OUT017               1000
4              FDY38            OUT027               1000
...              ...               ...                ...
5676           FDB58            OUT046               1000
5677           FDD47            OUT018               1000
5678           NCO17            OUT045               1000
5679           FDJ26            OUT017               1000
5680           FDU37            OUT045               1000

[5681 rows x 3 columns]

In [78]:
df_sub['Item_Outlet_Sales']= final_pred.copy()

In [82]:
df_sub.to_csv("Submissions/submission_8.csv", index= False)